In [2]:
#@title ***Getting drive content***

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [114]:
#@title ***Importing required librarys***

import math
import numpy as np
import cv2 as cv
import glob
from tensorflow import keras
from sklearn import svm
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import joblib   

In [4]:
#@title ***Project path***

project_path = "/content/drive/MyDrive/Colab Notebooks/Ml_Project"

In [112]:
#@title ***Reading images utility***

def read_images(gray_scale=False):
  print('Reading images...')
  images = []
  labels = []

  for label in range(0, 10):
    path = f"{project_path}/Dataset/{label}/*.JPG"
      
    for img_path in glob.glob(path):
      # reading image
      img = cv.imread(img_path)
      # resize image 
      img = cv.resize(img, (100, 100))
      
      if gray_scale:
        # convert image from BGR to gary
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
      else:
        # convert image from BGR to RGB
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)       
        
      images.append(img)
      labels.append(label)

  images = np.array(images)
  labels = np.array(labels)
    
    
  # normalize images
  if not gray_scale:
    # calculate average
    avg = np.average(images, axis=0)
    # subtract averages from each image.
    images = images - avg
      
  # divide each image by 255
  images = images / 255.0 

  print('Images read successfully.\n')

  return (images, labels)

In [68]:
#@title ***Get model measurements utility***

def get_measurements(true_y, pred_y, average='micro'):
  return {
      "accuracy": accuracy_score(true_y, pred_y),
      "recall": recall_score(true_y, pred_y, average=average),
      "precision": precision_score(true_y, pred_y, average=average),
      "fscore": f1_score(true_y, pred_y, average=average),
  }


In [122]:
#@title ***Running model with cross-validation utility***

def run_model_with_cv(model_id, clf, images, labels, train_size=0.8, k=5):
  # Spliting data into training & testing
  training_images, testing_images, training_labels, testing_labels =  train_test_split(images, labels, train_size=train_size)

  scoring=('accuracy', 'recall_micro', 'precision_micro', 'f1_micro')

  cv = cross_validate(clf, training_images, training_labels, scoring=scoring, verbose=k, cv=k, return_train_score=True, return_estimator=True)

  best_model_accuracy, best_model_idx  = 0, -1

  with open(f"{project_path}/{model_id}.txt", 'w') as file:
    file.write(f"Model: {clf}\n\n\n")

    file.write("Training\n")
    file.write("-------------\n\n")
    for key in cv.keys():
      if key == 'estimator':
        continue
      
      file.write(f"{key}: {cv[key]}\n\n")

    file.write("\n\n")

    file.write("Testing\n")
    file.write("-------------\n\n")
    for (idx, model) in enumerate(cv['estimator']):
      # making prediction
      predicted_labels = model.predict(testing_images)

      measurements = get_measurements(testing_labels, predicted_labels)

      if measurements['accuracy'] > best_model_accuracy:
        best_model_accuracy = measurements['accuracy']
        best_model_idx = idx

      # reporting model
      file.write(f"model#{idx+1}\n")
      file.write(f"accuracy: {measurements['accuracy']}\n")
      file.write(f"recall: {measurements['recall']}\n")
      file.write(f"precision: {measurements['precision']}\n")
      file.write(f"fscore: {measurements['fscore']}\n\n")

  # Saving model
  joblib.dump(cv['estimator'][best_model_idx], f"{project_path}/{model_id}.pkl")

In [130]:
#@title ***Running NN model utility***

def run_nn_model(model_id, model: keras.models.Sequential, images, labels, train_size=0.8, epochs=30, k=5):
  # Reporting model summary
  with open(f"{project_path}/{model_id}.txt", "w") as file:
    model.summary(print_fn=lambda x: file.write(x + '\n'))
    file.write('\n\n')

  # Spliting data into training & testing
  training_images, testing_images, training_labels, testing_labels =  train_test_split(images, labels, train_size=train_size)

  # Configuring model
  model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
  )

  best_model_acc = 0

  for i, (train_idx, val_idx) in enumerate(StratifiedKFold(n_splits=k).split(training_images, training_labels)):
    print("\n***********************************")
    print(f"*************** k#{i+1} ***************")
    print("***********************************\n")

    # Fitting model
    model.fit(
      training_images[train_idx], 
      training_labels[train_idx], 
      epochs=epochs, 
      verbose=1,
      validation_data=(training_images[val_idx], training_labels[val_idx]),
    )

    # Evaluating model
    predicted_labels = model.predict(testing_images, verbose=1)
    measurements = get_measurements(testing_labels, np.argmax(predicted_labels, axis=1))

    # Saving model
    if measurements['accuracy'] > best_model_acc:
      best_model_acc = measurements['accuracy']
      model.save(f"{project_path}/{model_id}.h5")

    # Reporting model
    with open(f"{project_path}/{model_id}.txt", "a") as file: 
      file.write(f"k#{i+1}\n")
      file.write(f"accuracy = {measurements['accuracy']}\n")
      file.write(f"recall = {measurements['recall']}\n")
      file.write(f"precision = {measurements['precision']}\n")
      file.write(f"fscore = {measurements['fscore']}\n\n")

In [131]:
#@title ***Model 1***

# Reading images & labels
images, labels = read_images(True)

# Setting model up
model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=(100, 100)),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(10, activation='softmax')
])

# Running model
run_nn_model('model1', model, images, labels)

Reading images...
Images read successfully.


***********************************
*************** k#1 ***************
***********************************

Epoch 1/30
42/42 [==============================] - 4s 87ms/step - loss: 7.8779 - accuracy: 0.1099 - val_loss: 2.5793 - val_accuracy: 0.1000
Epoch 2/30
42/42 [==============================] - 3s 79ms/step - loss: 2.3311 - accuracy: 0.1486 - val_loss: 2.2967 - val_accuracy: 0.1303
Epoch 3/30
42/42 [==============================] - 3s 81ms/step - loss: 2.1472 - accuracy: 0.2350 - val_loss: 2.1644 - val_accuracy: 0.1636
Epoch 4/30
42/42 [==============================] - 3s 78ms/step - loss: 1.9408 - accuracy: 0.3624 - val_loss: 1.8611 - val_accuracy: 0.2939
Epoch 5/30
42/42 [==============================] - 3s 78ms/step - loss: 1.7191 - accuracy: 0.4102 - val_loss: 1.6379 - val_accuracy: 0.4303
Epoch 6/30
42/42 [==============================] - 3s 78ms/step - loss: 1.5275 - accuracy: 0.4799 - val_loss: 1.4459 - val_accuracy: 0.5182

In [132]:
#@title ***Model 2***

# Reading images & labels
images, labels = read_images(True)

# Setting model up
model = keras.models.Sequential([
  keras.layers.Flatten(input_shape=(100, 100)),
  keras.layers.Dense(10000, activation='relu'),
  keras.layers.Dense(10, activation='softmax')
])

# Running model
run_nn_model('model2', model, images, labels)

Reading images...
Images read successfully.


***********************************
*************** k#1 ***************
***********************************

Epoch 1/30
42/42 [==============================] - 27s 626ms/step - loss: 33.9992 - accuracy: 0.1145 - val_loss: 3.4759 - val_accuracy: 0.1091
Epoch 2/30
42/42 [==============================] - 26s 618ms/step - loss: 2.4697 - accuracy: 0.2039 - val_loss: 2.3138 - val_accuracy: 0.1970
Epoch 3/30
42/42 [==============================] - 26s 625ms/step - loss: 2.0143 - accuracy: 0.3093 - val_loss: 1.8299 - val_accuracy: 0.4030
Epoch 4/30
42/42 [==============================] - 30s 721ms/step - loss: 1.8106 - accuracy: 0.3730 - val_loss: 1.8695 - val_accuracy: 0.4061
Epoch 5/30
42/42 [==============================] - 28s 654ms/step - loss: 1.6090 - accuracy: 0.4716 - val_loss: 2.3318 - val_accuracy: 0.1848
Epoch 6/30
42/42 [==============================] - 28s 665ms/step - loss: 1.5151 - accuracy: 0.4882 - val_loss: 1.5863 - val_acc

In [133]:
#@title ***Model 3***

# Reading images & labels
images, labels = read_images()

# Setting model up
model = keras.models.Sequential([
  keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
  keras.layers.MaxPool2D((2, 2)),
  keras.layers.Conv2D(64, (3, 3), activation='relu'),
  keras.layers.MaxPool2D((2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(512, activation='relu'),
  keras.layers.Dense(10, activation='softmax')
])

# Running model
run_nn_model('model3', model, images, labels)

Reading images...
Images read successfully.


***********************************
*************** k#1 ***************
***********************************

Epoch 1/30
42/42 [==============================] - 38s 892ms/step - loss: 1.3832 - accuracy: 0.5254 - val_loss: 0.7659 - val_accuracy: 0.7394
Epoch 2/30
42/42 [==============================] - 34s 806ms/step - loss: 0.4836 - accuracy: 0.8370 - val_loss: 0.5308 - val_accuracy: 0.8152
Epoch 3/30
42/42 [==============================] - 33s 796ms/step - loss: 0.1765 - accuracy: 0.9424 - val_loss: 0.3707 - val_accuracy: 0.8788
Epoch 4/30
42/42 [==============================] - 37s 883ms/step - loss: 0.0595 - accuracy: 0.9848 - val_loss: 0.4609 - val_accuracy: 0.8788
Epoch 5/30
42/42 [==============================] - 34s 818ms/step - loss: 0.0162 - accuracy: 0.9962 - val_loss: 0.6453 - val_accuracy: 0.8576
Epoch 6/30
42/42 [==============================] - 36s 871ms/step - loss: 0.0153 - accuracy: 0.9955 - val_loss: 0.5073 - val_accu

In [123]:
#@title ***Model 4***

# Reading images & labels
images, labels = read_images()

# Flatting images
images = np.array([img.flatten() for img in images])

# model classifier
model = svm.SVC(C=10, gamma=0.001)

# Running model
run_model_with_cv('model4', model, images, labels)

Reading images...
Images read successfully.



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END  accuracy: (train=1.000, test=0.891) f1_micro: (train=1.000, test=0.891) precision_micro: (train=1.000, test=0.891) recall_micro: (train=1.000, test=0.891) total time= 1.6min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s


[CV] END  accuracy: (train=1.000, test=0.861) f1_micro: (train=1.000, test=0.861) precision_micro: (train=1.000, test=0.861) recall_micro: (train=1.000, test=0.861) total time= 1.5min


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.6min remaining:    0.0s


[CV] END  accuracy: (train=1.000, test=0.827) f1_micro: (train=1.000, test=0.827) precision_micro: (train=1.000, test=0.827) recall_micro: (train=1.000, test=0.827) total time= 1.4min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 11.2min remaining:    0.0s


[CV] END  accuracy: (train=1.000, test=0.842) f1_micro: (train=1.000, test=0.842) precision_micro: (train=1.000, test=0.842) recall_micro: (train=1.000, test=0.842) total time= 1.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 13.7min finished
